In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import setup, compare_models, blend_models,tune_model,predict_model,get_config, finalize_model

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

from workalendar.asia import SouthKorea
import pendulum

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
test

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치
5,2021-02-03,수,2924,59,199,5,286.0,모닝롤/게살모닝샌드 우유/주스 계란후라이/찐계란 소고기죽/흑미밥 시래기된장국 베이컨...,쌀밥/흑미밥/찰현미밥 아욱국 매콤해물볶음 감자조림 미나리나물 포기김치 콥샐러드*렌치D,오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치
6,2021-02-04,목,2924,61,211,476,288.0,모닝롤/사과파이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 아욱국 새송이버섯곤약장...,쌀밥/차조밥/찰현미밥 설렁탕 고등어김치말이찜 볼어묵굴소스볶음 브로콜리숙회*초장 석박...,흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치
7,2021-02-05,금,2924,169,252,0,256.0,모닝롤/앙버터모닝빵 우유/주스 계란후라이/찐계란 고구마죽/흑미밥 옹심이국 머위나물무...,쌀밥/흑미밥/찰현미밥 북엇국 닭볶음탕 채소전*장 솎음열무나물무침 포기김치 양상추샐러...,유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치
8,2021-02-08,월,2924,88,174,690,329.0,모닝롤/스콘 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 꽃게탕 근대나물무침 연두부...,쌀밥/흑미밥/찰현미밥 감자양파국 돈수육*씨앗쌈장 매콤어묵볶음 콩나물파채무침 포기김치...,흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치
9,2021-02-09,화,2924,94,183,542,329.0,모닝롤/치즈팡샌드 우유/주스 계란후라이/찐계란 팥죽/흑미밥 맑은버섯국 시금치나물무침...,쌀밥/기장밥/찰현미밥 장각백숙 적어양념장구이 채소스틱*쌈장 도라지오이초무침 겉절이김...,흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지


## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [33]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['day2']=0
train.loc[train['일']>9, 'day2'] = 1
train.loc[train['일']>19, 'day2'] = 2

test['day2']=0
test.loc[test['일']>9, 'day2'] = 1
test.loc[test['일']>19, 'day2'] = 2

In [5]:
weekday = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5
}

train['요일'] = train['요일'].map(weekday)
test['요일'] = test['요일'].map(weekday)

In [6]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,월,주,일,출근,휴가비율,출장비율,야근비율,재택비율
0,2016-02-01,1,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2,5,1,2401.0,0.019223,0.057670,0.099125,0.0
1,2016-02-02,2,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2,5,2,2378.0,0.019223,0.066513,0.134146,0.0
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2,5,3,2365.0,0.021530,0.069204,0.046934,0.0
3,2016-02-04,4,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2,5,4,2277.0,0.039985,0.084583,0.155907,0.0
4,2016-02-05,5,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2,5,5,2142.0,0.106882,0.069589,0.015873,0.0


### 요일 레이블 인코딩

In [7]:
weekday_rank4dinner = {
    1: 1,
    2: 2,
    3: 5,
    4: 3,
    5: 4,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일']
test['요일(중식)'] = test['요일']

### 월 레이블 인코딩

In [8]:
month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

In [9]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 8,
    6: 6,
    7: 9,
    8: 7,
    9: 5,
    10: 3,
    11: 10,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

## 주 EDA

In [10]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

## 공휴일 변수 생성

In [11]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 'S'
    if tomorrow in holidays:
        return 'T'
    elif yesterday in holidays:
        return 'Y'
    else : 
        return 'N'

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

## 텍스트(메뉴) 전처리

### train/test에서 중복 메뉴만 변수로서 사용

In [12]:
menu_train = train[['조식메뉴','중식메뉴','석식메뉴']]
menu_test = test[['조식메뉴','중식메뉴','석식메뉴']]
print(menu_train.shape)
print(menu_test.shape)

(1205, 3)
(50, 3)


In [15]:
def clean_split(df):
    df = df.split()
    for j in df:
        if '(' in j or '쌀밥' in df or '김치' in df:
            del df[df.index(j)]    
            
    return df

In [17]:
def make_list_to_string(menu_list):
    menu_str_list = []
    for menu in menu_list:
        if len(menu) == 0:
            continue

        string = " ".join(menu)

        menu_str_list.append(string)
    return menu_str_list

In [28]:
menu_train['중식메뉴_prepro'] = menu_train['중식메뉴'].apply(clean_split)
menu_train['석식메뉴_prepro'] = menu_train['석식메뉴'].apply(clean_split)

menu_test['중식메뉴_prepro'] = menu_test['중식메뉴'].apply(clean_split)
menu_test['석식메뉴_prepro'] = menu_test['석식메뉴'].apply(clean_split)

lunch_train= menu_train.중식메뉴_prepro.tolist()
lunch_test= menu_test.중식메뉴_prepro.tolist()
dinner_train = menu_train.석식메뉴_prepro.tolist()
dinner_test = menu_test.석식메뉴_prepro.tolist()

lunch_list_train  = make_list_to_string(lunch_train)
lunch_list_test   = make_list_to_string(lunch_test)
dinner_list_train = make_list_to_string(dinner_train)
dinner_list_test  = make_list_to_string(dinner_test)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
lunch_matrix_train = vectorizer.fit_transform(lunch_list_train)
lunch_df_train = pd.DataFrame(lunch_matrix_train.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
dinner_matrix_train = vectorizer.fit_transform(dinner_list_train)
dinner_df_train = pd.DataFrame(dinner_matrix_train.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
lunch_matrix_test = vectorizer.fit_transform(lunch_list_test)
lunch_df_test = pd.DataFrame(lunch_matrix_test.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
dinner_matrix_test = vectorizer.fit_transform(dinner_list_test)
dinner_df_test = pd.DataFrame(dinner_matrix_test.toarray(), columns=vectorizer.get_feature_names())

In [30]:
# 중복 메뉴 선정
lunch_intersection = list(set(lunch_df_train.columns) & set(lunch_df_test.columns))
print(len(lunch_intersection))
dinner_intersection = list(set(dinner_df_train.columns) & set(dinner_df_test.columns))
print(len(dinner_intersection))

167
120


## 최종 데이터 셋 구축

In [75]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '요일(석식)', '요일(중식)', '월(중식)',
       '월(석식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차'],
      dtype='object')

In [31]:
lunch_df_f = pd.concat([train[['공휴일전후', '몇주차', '요일(중식)','월(중식)','day2','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','중식계']],lunch_df_train[lunch_intersection]],axis=1)
lunch_df_test_f = pd.concat([test[['공휴일전후', '몇주차', '요일(중식)','월(중식)','day2','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수']],lunch_df_test[lunch_intersection]],axis=1)


dinner_df_f = pd.concat([train[['공휴일전후', '몇주차', '요일(석식)','월(석식)','day2','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','석식계']],dinner_df_train[dinner_intersection]],axis=1)
dinner_df_test_f = pd.concat([test[['공휴일전후', '몇주차', '요일(석식)','월(석식)','day2','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수']],dinner_df_test[dinner_intersection]],axis=1)

In [79]:
print(lunch_df_f.shape)
print(lunch_df_test_f.shape)

(1205, 160)
(50, 159)


In [80]:
print(dinner_df_f.shape)
print(dinner_df_test_f.shape)

(1205, 123)
(50, 122)


## OLS

In [86]:
from statsmodels.api import OLS

In [88]:
lunch_df_f['공휴일전후'] = lunch_df_f['공휴일전후'].apply(lambda x: 1 if x=='Y' else 0)

In [90]:
ols = OLS(lunch_df_f['중식계'],lunch_df_f.drop(['중식계'],axis=1))
result = ols.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    중식계   R-squared (uncentered):                   0.988
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              550.5
Date:                Thu, 01 Jul 2021   Prob (F-statistic):                        0.00
Time:                        11:00:13   Log-Likelihood:                         -7260.3
No. Observations:                1205   AIC:                                  1.483e+04
Df Residuals:                    1048   BIC:                                  1.563e+04
Df Model:                         157                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
공휴일전후        127.3596     22.170      5.745      0.000      83.856     170.863
몇주차          -21.9958     11.866     -1.854      0.064     -45.280       1.289
요일(중식)       -70.2689      4.089    -17.186      0.000     -78.292     -62.246
월(중식)         -0.8451      1.303     -0.648      0.517      -3.403       1.713
일              2.1326      1.742      1.224      0.221      -1.286       5.551
주(중식)          1.5058      0.323      4.663      0.000       0.872       2.139
출근             0.5276      0.014     38.306      0.000       0.501       0.555
휴가비율          38.3350    917.165      0.042      0.967   -1761.354    1838.024
출장비율        1.902e+04   1001.124     18.995      0.000    1.71e+04     2.1e+04
야근비율         230.5471     43.995      5.240      0.000     144.218     316.876
재택비율         963.4084    134.225      7.178      0.000     700.029    1226.788
본사출장자수        -7.6187      0.391    -19.467      0.000      -8.387      -6.851
본사휴가자수         0.0189      0.326      0.058      0.954      -0.621       0.659
콥샐러드          44.9603    135.500      0.332      0.740    -220.922     310.843
브로콜리숙회       -77.0358    214.436     -0.359      0.719    -497.808     343.736
매운돈갈비찜      -104.5467    263.037     -0.397      0.691    -620.685     411.592
세발나물무침        89.4907    197.436      0.453      0.650    -297.923     476.905
콩나물불고기       258.9419    156.178      1.658      0.098     -47.516     565.399
두부양념조림        68.4386     74.206      0.922      0.357     -77.170     214.047
초장          -123.3487     77.892     -1.584      0.114    -276.191      29.494
동태매운탕       -352.2251    166.532     -2.115      0.035    -679.000     -25.451
채소스틱        -240.3332    305.189     -0.787      0.431    -839.184     358.518
파인d          328.5577    193.634      1.697      0.090     -51.396     708.512
가지나물          22.4107     60.671      0.369      0.712     -96.640     141.461
쌈채소          -23.6717    267.783     -0.088      0.930    -549.123     501.780
골뱅이채소무침       44.2213    297.193      0.149      0.882    -538.939     627.382
호박고추장찌개      254.3598    249.983      1.018      0.309    -236.165     744.884
열무된장국          6.1208     57.202      0.107      0.915    -106.122     118.363
매실d          104.3881    123.695      0.844      0.399    -138.331     347.107
쇠고기숙주볶음      143.9087     86.116      1.671      0.095     -25.072     312.889
씨앗쌈장         -29.4273    149.729     -0.197      0.844    -323.230     264.376
동파육            8.0428     90.455      0.089      0.929    -169.452     185.537
삼색유자청무침      420.4847    237.832      1.768      0.077     -46.197     887.166
소불고기         198.4149     63.641      3.118      0.002      73.537     323.293
유부채소겨자냉채     249.3283    308.796      0.807      0.420    -35

In [101]:
vari_df = pd.DataFrame(result.pvalues)
vari_df[vari_df[0]<0.1].index

Index(['공휴일전후', '몇주차', '요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
       '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
       '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d'],
      dtype='object')

In [108]:
ols = OLS(lunch_df_f['중식계'],lunch_df_f[['공휴일전후', '몇주차', '요일(중식)', '주(중식)','day2', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
       '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
       '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d','휴가비율','월(중식)']])
result = ols.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    중식계   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.987
Method:                 Least Squares   F-statistic:                              2627.
Date:                Thu, 01 Jul 2021   Prob (F-statistic):                        0.00
Time:                        11:08:02   Log-Likelihood:                         -7306.9
No. Observations:                1205   AIC:                                  1.468e+04
Df Residuals:                    1171   BIC:                                  1.486e+04
Df Model:                          34                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
공휴일전후        116.5972     20.200      5.772      0.000      76.966     156.229
몇주차          -27.2085     10.728     -2.536      0.011     -48.257      -6.160
요일(중식)       -71.7595      3.768    -19.047      0.000     -79.151     -64.368
주(중식)          1.5880      0.297      5.342      0.000       1.005       2.171
출근             0.5318      0.012     42.573      0.000       0.507       0.556
출장비율        1.931e+04    681.438     28.331      0.000     1.8e+04    2.06e+04
야근비율         218.3177     38.695      5.642      0.000     142.398     294.237
재택비율        1092.7775    114.411      9.551      0.000     868.303    1317.252
본사출장자수        -7.7631      0.280    -27.690      0.000      -8.313      -7.213
콩나물불고기       295.4787    137.477      2.149      0.032      25.751     565.207
동태매운탕       -327.6254    148.774     -2.202      0.028    -619.519     -35.732
파인d          195.1716    168.345      1.159      0.247    -135.120     525.463
쇠고기숙주볶음      115.2167     74.788      1.541      0.124     -31.517     261.951
삼색유자청무침      387.1791    167.947      2.305      0.021      57.669     716.689
소불고기         205.3712     58.805      3.492      0.000      89.996     320.747
바나나          293.1034    120.220      2.438      0.015      57.234     528.973
황도d          166.0496    125.053      1.328      0.184     -79.303     411.403
콩나물파채무침      229.4234    108.879      2.107      0.035      15.804     443.043
버섯매운탕       -315.7564    142.593     -2.214      0.027    -595.524     -35.989
돈육간장불고기      145.1698     79.727      1.821      0.069     -11.253     301.593
양상추샐러드       236.5101     65.502      3.611      0.000     107.996     365.024
석박지          137.4225     82.687      1.662      0.097     -24.810     299.655
수박           170.8075     93.379      1.829      0.068     -12.401     354.016
오리대패불고기     -245.3582    119.613     -2.051      0.040    -480.037     -10.679
오이생채        -203.5390     97.860     -2.080      0.038    -395.539     -11.539
흑임자d        -383.5123    183.271     -2.093      0.037    -743.088     -23.937
고등어구이       -170.8129     73.915     -2.311      0.021    -315.833     -25.793
닭볶음탕        -190.8826     56.268     -3.392      0.001    -301.280     -80.485
청경채찜         752.0508    266.759      2.819      0.005     228.673    1275.429
우렁된장찌개      -401.4009    179.256     -2.239      0.025    -753.100     -49.702
요거트d         107.3778     82.140      1.307      0.191     -53.781     268.536
휴가비율         106.7835     70.745      1.509      0.131     -32.017     245.584
월(중식)         -0.0106      1.216     -0.009      0.993      -2.397       2.376
일              2.9697      1.584      1.875      0.061      -0.138       6.077
=============================================================

# 중식 예측모델

In [187]:
num_feature = ['요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
       '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
       '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d','휴가비율','중식계']
num_feature.remove('중식계')


lunch_regression_model = setup(data=lunch_df_f[['공휴일전후', '몇주차', '요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
       '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
       '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d','휴가비율','중식계']],
                               target='중식계', train_size=0.8,n_jobs=-1,fold = 8,
                               numeric_features=num_feature,
                              numeric_imputation = 'mean',
                                normalize = True)

,Description,Value
0,session_id,6399
1,Target,중식계
2,Original Data,"(1205, 33)"
3,Missing Values,False
4,Numeric Features,30
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 37)"


In [188]:
# 최고 성능 모델 선정(2개)
lunch_regression_best_models = compare_models(fold=8, n_select=2, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,67.0155,7923.2444,88.4284,0.8174,0.1126,0.0827,0.5525
gbr,Gradient Boosting Regressor,69.0013,8394.4705,91.1789,0.8061,0.1154,0.0850,0.0288
xgboost,Extreme Gradient Boosting,71.7655,8891.1636,93.8233,0.7946,0.1186,0.0881,0.1725
lightgbm,Light Gradient Boosting Machine,73.1150,9155.3637,95.3365,0.7886,0.1207,0.0898,0.0462
rf,Random Forest Regressor,71.6951,9179.4869,95.4181,0.7880,0.1210,0.0884,0.0712
et,Extra Trees Regressor,74.4429,9887.0650,98.9732,0.7718,0.1251,0.0918,0.0588
lr,Linear Regression,83.9212,11758.3230,108.0945,0.7271,0.1322,0.1010,0.0050
ridge,Ridge Regression,83.8402,11765.2415,108.1322,0.7269,0.1322,0.1009,0.0050
br,Bayesian Ridge,83.9903,11834.7816,108.4839,0.7255,0.1327,0.1013,0.0050
ada,AdaBoost Regressor,84.7400,11907.8098,108.7769,0.7241,0.1363,0.1046,0.0225


In [190]:
lunch_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in lunch_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,78.5040,10109.1134,100.5441,0.7667,0.1162,0.0897
1,68.4174,8123.0522,90.1280,0.8160,0.1168,0.0846
2,70.8492,7959.9345,89.2185,0.8154,0.1098,0.0883
3,61.7180,6655.1047,81.5788,0.8162,0.1060,0.0793
4,75.4573,9531.1897,97.6278,0.7795,0.1167,0.0880
5,56.9673,5446.6771,73.8016,0.8802,0.0963,0.0713
6,69.3546,8843.4669,94.0397,0.7819,0.1250,0.0881
7,81.2779,11953.4289,109.3317,0.7577,0.1527,0.1066
Mean,70.3182,8577.7459,92.0338,0.8017,0.1174,0.0870
SD,7.6695,1897.6487,10.3697,0.0368,0.0156,0.0094


In [191]:
lunch_model_f = blend_models(estimator_list=lunch_regression_best_models_tuned, fold=5, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,75.4227,9115.4912,95.4751,0.7972,0.1169,0.0890
1,69.1350,8092.9916,89.9611,0.8067,0.1078,0.0841
2,67.4073,7562.6469,86.9635,0.8122,0.1084,0.0830
3,66.2534,8069.0391,89.8278,0.8179,0.1074,0.0787
4,72.2933,10051.8447,100.2589,0.7779,0.1426,0.0957
Mean,70.1024,8578.4027,92.4973,0.8024,0.1166,0.0861
SD,3.3502,892.8976,4.7601,0.0140,0.0135,0.0058


In [192]:
pred = predict_model(lunch_model_f)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,69.7131,8534.8340,92.3842,0.8088,0.1217,0.0888


### 테스트 데이터 예측

In [193]:
submission = pd.read_csv('../data/sample_submission.csv')

In [194]:
final_model_lunch = finalize_model(lunch_model_f)
prep_pipe_lunch = get_config('prep_pipe')
prep_pipe_lunch.steps.append(['trained_model', final_model_lunch])

In [195]:
pred_lunch = prep_pipe_lunch.predict(lunch_df_test_f)

submission.iloc[:,1] = pred_lunch
submission.head()

,일자,중식계,석식계
0,2021-01-27,1110.809420,0
1,2021-01-28,1075.043315,0
2,2021-01-29,711.250147,0
3,2021-02-01,1303.682363,0
4,2021-02-02,1132.213975,0


# 석식 예측모델

## OLS

In [141]:
dinner_df_f['공휴일전후'] = dinner_df_f['공휴일전후'].apply(lambda x: 1 if x=='Y' else 0)

In [143]:
ols = OLS(dinner_df_f['석식계'],dinner_df_f.drop(['석식계'],axis=1))
result = ols.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    석식계   R-squared (uncentered):                   0.960
Model:                            OLS   Adj. R-squared (uncentered):              0.955
Method:                 Least Squares   F-statistic:                              222.8
Date:                Thu, 01 Jul 2021   Prob (F-statistic):                        0.00
Time:                        11:23:09   Log-Likelihood:                         -7222.0
No. Observations:                1205   AIC:                                  1.468e+04
Df Residuals:                    1089   BIC:                                  1.527e+04
Df Model:                         116                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
공휴일전후         19.9592     20.059      0.995      0.320     -19.399      59.317
몇주차          -32.2063      9.970     -3.230      0.001     -51.769     -12.643
요일(석식)       -16.4004      4.234     -3.874      0.000     -24.708      -8.093
월(석식)         -3.1934      1.148     -2.781      0.006      -5.447      -0.940
일              2.8916      1.501      1.927      0.054      -0.053       5.836
주(석식)          1.6563      0.289      5.724      0.000       1.089       2.224
출근             0.2061      0.014     15.041      0.000       0.179       0.233
휴가비율        -325.2108    848.242     -0.383      0.702   -1989.584    1339.162
출장비율        9460.3987    935.039     10.118      0.000    7625.717    1.13e+04
야근비율         544.9718     61.817      8.816      0.000     423.677     666.266
재택비율         207.2652    127.977      1.620      0.106     -43.843     458.374
본사출장자수        -3.4465      0.356     -9.693      0.000      -4.144      -2.749
본사휴가자수         0.1536      0.301      0.510      0.610      -0.438       0.745
흑미밥          -68.5808     64.970     -1.056      0.291    -196.061      58.900
세발나물무침       172.3431    178.428      0.966      0.334    -177.759     522.445
쇠고기규동덮밥       -6.1119    222.790     -0.027      0.978    -443.258     431.034
깐풍육           75.3846     87.930      0.857      0.391     -97.148     247.917
달래된장찌개       -41.1203    178.364     -0.231      0.818    -391.097     308.856
꽃맛살샐러드        -5.8886     51.349     -0.115      0.909    -106.642      94.865
라볶이         -214.1810    197.592     -1.084      0.279    -601.885     173.523
미나리전         -88.1349     73.784     -1.195      0.233    -232.910      56.640
단무지무침         14.1766     57.593      0.246      0.806     -98.830     127.183
쇠고기숙주볶음       29.5333     92.433      0.320      0.749    -151.834     210.901
김칫국           60.7611    103.745      0.586      0.558    -142.801     264.324
쇠고기우엉볶음     -147.7309    218.369     -0.677      0.499    -576.202     280.740
오징어어묵무침       33.9847    228.492      0.149      0.882    -414.349     482.318
오이무침         -23.8335     60.955     -0.391      0.696    -143.435      95.768
연근조림        -118.7948    319.621     -0.372      0.710    -745.938     508.348
쑥갓나물         -12.7652    197.589     -0.065      0.949    -400.464     374.933
취나물무침         73.8623    240.617      0.307      0.759    -398.263     545.988
순두부찌개         81.3932     55.428      1.468      0.142     -27.364     190.151
황태국           90.7203     79.474      1.142      0.254     -65.219     246.660
참나물초장무침      183.2916    174.788      1.049      0.295    -159.669     526.252
순대찜           49.8317    252.621      0.197      0.844    -445.847     545.510
삼치엿장구이      -184.9386    196.903     -0.939      0.348    -57

In [146]:
vari_df = pd.DataFrame(result.pvalues)
vari_df[vari_df[0]<0.11].index

Index(['몇주차', '요일(석식)', '월(석식)', '일', '주(석식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '오므라이스'],
      dtype='object')

In [151]:
ols = OLS(dinner_df_f['석식계'],dinner_df_f[['몇주차', '요일(석식)', '월(석식)', 'day2', '주(석식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '오므라이스','휴가비율']])
result = ols.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    석식계   R-squared (uncentered):                   0.958
Model:                            OLS   Adj. R-squared (uncentered):              0.957
Method:                 Least Squares   F-statistic:                              2242.
Date:                Thu, 01 Jul 2021   Prob (F-statistic):                        0.00
Time:                        11:27:06   Log-Likelihood:                         -7251.6
No. Observations:                1205   AIC:                                  1.453e+04
Df Residuals:                    1193   BIC:                                  1.459e+04
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
몇주차          -30.5709      9.220     -3.316      0.001     -48.661     -12.481
요일(석식)       -14.7969      3.867     -3.827      0.000     -22.383      -7.210
월(석식)         -3.2294      1.067     -3.026      0.003      -5.323      -1.136
일              2.7846      1.389      2.004      0.045       0.059       5.510
주(석식)          1.5937      0.268      5.939      0.000       1.067       2.120
출근             0.2099      0.012     16.997      0.000       0.186       0.234
출장비율        9406.3628    650.435     14.462      0.000    8130.238    1.07e+04
야근비율         550.2146     54.433     10.108      0.000     443.420     657.009
재택비율         109.3714     96.513      1.133      0.257     -79.982     298.725
본사출장자수        -3.4734      0.252    -13.785      0.000      -3.968      -2.979
오므라이스        100.3730     56.964      1.762      0.078     -11.387     212.133
휴가비율         105.3706     57.897      1.820      0.069      -8.221     218.963
==============================================================================
Omnibus:                      356.365   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1286.837
Skew:                          -1.405   Prob(JB):                    3.69e-280
Kurtosis:                       7.212   Cond. No.                     5.40e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 5.4e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [196]:
num_feature = ['요일(석식)', '월(석식)', '주(석식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '오므라이스','휴가비율','석식계']
num_feature.remove('석식계')

dinner_regression_model = setup(data=dinner_df_f[['몇주차', '요일(석식)', '월(석식)', '주(석식)', '출근', '출장비율', '야근비율', '재택비율',
       '본사출장자수', '오므라이스','휴가비율','석식계','공휴일전후']], target='석식계', train_size=0.8,n_jobs=-1,fold = 8,
                                                              numeric_features=num_feature,
                                numeric_imputation = 'mean',
#                                 categorical_features = ['일'],
                                normalize = True)

,Description,Value
0,session_id,7974
1,Target,석식계
2,Original Data,"(1205, 13)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 17)"


In [197]:
dinner_regression_best_models = compare_models(fold=8, n_select=2, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,58.6245,7288.0850,84.5188,0.6274,0.9372,0.1255,0.0250
catboost,CatBoost Regressor,57.1630,7298.8441,84.4557,0.6257,0.9041,0.1247,0.5312
rf,Random Forest Regressor,58.8045,7669.9813,86.0899,0.6051,0.9064,0.1302,0.0675
et,Extra Trees Regressor,58.3041,7805.0910,86.3845,0.6034,0.8370,0.1302,0.0500
lightgbm,Light Gradient Boosting Machine,60.9992,7674.0440,86.6383,0.6023,0.9291,0.1329,0.0512
xgboost,Extreme Gradient Boosting,59.3517,7744.8915,86.8601,0.6006,0.8827,0.1300,0.1875
knn,K Neighbors Regressor,65.9851,9153.7434,94.4977,0.5319,0.9598,0.1464,0.0062
lr,Linear Regression,70.7832,10288.1361,100.4063,0.4800,1.0756,0.1449,0.0050
ridge,Ridge Regression,70.8192,10279.8063,100.3888,0.4800,1.0753,0.1448,0.0050
br,Bayesian Ridge,70.7984,10289.0243,100.4275,0.4797,1.0761,0.1447,0.0050


In [198]:
dinner_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in dinner_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,60.9554,8626.3605,92.8782,0.6578,1.2823,0.1480
1,52.5183,6177.8364,78.5992,0.6950,1.0445,0.1071
2,66.3159,8844.3241,94.0443,0.5918,1.0252,0.1461
3,50.7841,4887.8635,69.9133,0.6471,0.6590,0.1101
4,53.2911,7370.4641,85.8514,0.5529,0.8398,0.1157
5,45.5632,3481.7735,59.0066,0.7547,0.3631,0.0989
6,63.1936,8334.7771,91.2950,0.7050,1.2293,0.1297
7,62.7452,10122.1561,100.6089,0.4250,1.0573,0.1359
Mean,56.9208,7230.6944,84.0246,0.6287,0.9375,0.1239
SD,6.8650,2085.7416,13.0599,0.0974,0.2860,0.0174


In [199]:
dinner_model_f = blend_models(estimator_list=dinner_regression_best_models, fold=8, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,61.3725,8730.7584,93.4385,0.6536,1.2860,0.1466
1,52.1693,6126.4539,78.2717,0.6975,1.0565,0.1043
2,67.9529,8629.4977,92.8951,0.6017,1.0743,0.1452
3,50.2704,4726.0339,68.7462,0.6588,0.6251,0.1092
4,51.1392,6582.6103,81.1333,0.6007,0.8538,0.1074
5,45.0128,3657.8534,60.4802,0.7423,0.2529,0.0996
6,65.3128,8531.7470,92.3675,0.6981,1.2105,0.1349
7,62.1265,9709.4351,98.5365,0.4484,1.0490,0.1355
Mean,56.9196,7086.7987,83.2336,0.6376,0.9260,0.1228
SD,7.7547,2017.2991,12.6082,0.0847,0.3189,0.0183


In [200]:
pred = predict_model(dinner_model_f)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,52.0126,5492.6969,74.1127,0.6865,1.0076,0.1012


## 테스트

### 테스트 데이터 예측

In [202]:
final_model_dinner = finalize_model(dinner_model_f)
prep_pipe_dinner = get_config('prep_pipe')
prep_pipe_dinner.steps.append(['trained_model', final_model_dinner])

In [203]:
pred_dinner = prep_pipe_dinner.predict(dinner_df_test_f)
submission.iloc[:,2] = pred_dinner
submission.head()

,일자,중식계,석식계
0,2021-01-27,1110.809420,371.137066
1,2021-01-28,1075.043315,403.254010
2,2021-01-29,711.250147,241.860023
3,2021-02-01,1303.682363,529.637671
4,2021-02-02,1132.213975,483.185102


# 저장

In [204]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_pycaret_(2).csv', index =False)

오늘 날짜 : 20210701
